In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_regression
import warnings
warnings.filterwarnings('ignore')

# Set style for visualizations
sns.set_style("whitegrid")
sns.set_palette("pastel")
plt.rcParams['figure.figsize'] = (12, 6)

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600 # Set high resolution for plots
mpl.rcParams['savefig.dpi'] = 600 # Set high resolution for saved plots

import plotly.io as pio
pio.renderers.default = 'notebook'  # Or 'notebook_connected' for JupyterLab

### 1. Data Loading & Initial Exploration

In [ ]:
# Load the dataset
df = pd.read_csv('insurance.csv')

# Display basic information
print(f"Dataset shape: {df.shape}")
print("\nFirst 5 rows:")
display(df.head())

print("\nData types and missing values:")
display(df.info())

print("\nDescriptive statistics:")
display(df.describe(include='all'))

print("\nUnique values in categorical columns:")
print(f"Sex: {df['sex'].unique()}")
print(f"Smoker: {df['smoker'].unique()}")
print(f"Region: {df['region'].unique()}")

### 2. Data Cleaning & Preprocessing


In [ ]:
# Make a copy of the original data
df_clean = df.copy()

# Check for missing values
print("Missing values before cleaning:")
print(df_clean.isnull().sum())

# Convert categorical columns to proper type
cat_cols = ['sex', 'smoker', 'region']
for col in cat_cols:
    df_clean[col] = df_clean[col].astype('category')

# Check for outliers in numerical columns
num_cols = ['age', 'bmi', 'children', 'charges']

plt.figure(figsize=(15, 10))
for i, col in enumerate(num_cols, 1):
    plt.subplot(2, 2, i)
    sns.boxplot(y=df_clean[col])
    plt.title(f'Boxplot of {col}')
plt.tight_layout()
plt.show()

# Handle outliers in charges (target variable)
Q1 = df_clean['charges'].quantile(0.25)
Q3 = df_clean['charges'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Lower bound for charges: {lower_bound}")
print(f"Upper bound for charges: {upper_bound}")

# We won't remove outliers in charges as they might represent genuine cases
# But we'll apply log transformation to reduce skewness
df_clean['log_charges'] = np.log1p(df_clean['charges'])

# Create age groups for better analysis
df_clean['age_group'] = pd.cut(df_clean['age'], 
                               bins=[0, 18, 30, 45, 60, 100],
                               labels=['0-18', '19-30', '31-45', '46-60', '60+'])

# Create BMI categories
df_clean['bmi_category'] = pd.cut(df_clean['bmi'],
                                 bins=[0, 18.5, 25, 30, 100],
                                 labels=['Underweight', 'Normal', 'Overweight', 'Obese'])

### 3. Exploratory Data Analysis (EDA)


In [ ]:
# Distribution of Charges (Original and Log Transformed)
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
sns.histplot(df_clean['charges'], bins=50, kde=True)
plt.title('Distribution of Insurance Charges')
plt.xlabel('Charges ($)')

plt.subplot(1, 2, 2)
sns.histplot(df_clean['log_charges'], bins=50, kde=True)
plt.title('Distribution of Log-Transformed Charges')
plt.xlabel('Log(Charges)')
plt.tight_layout()
plt.show()

# Charges by Categorical Features
cat_features = ['sex', 'smoker', 'region', 'age_group', 'bmi_category']
plt.figure(figsize=(18, 12))
for i, feature in enumerate(cat_features, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(x=feature, y='charges', data=df_clean)
    plt.title(f'Charges by {feature}')
    plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Correlation Heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df_clean[num_cols].corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap of Numerical Features')
plt.show()

# Pairplot of Numerical Features
sns.pairplot(df_clean[num_cols], diag_kind='kde')
plt.suptitle('Pairplot of Numerical Features', y=1.02)
plt.show()
 
# Create the interactive 3D scatter plot using Plotly
# 3D Scatter Plot of Age, BMI, and Charges
fig = px.scatter_3d(df_clean, x='age', y='bmi', z='charges',
                    color='smoker', size='children',
                    hover_data=['sex', 'region'],
                    title='3D Relationship: Age, BMI, and Charges')

# Interactive 3D Scatter Plot (using plotly)
fig = px.scatter_3d(df_clean, x='age', y='bmi', z='charges',
                    color='smoker', size='children',
                    hover_data=['sex', 'region'],
                    title='3D Relationship: Age, BMI, and Charges')
fig.show()

### 4. Statistical Analysis

In [ ]:
# Hypothesis Testing: Do smokers have significantly higher charges?
print("\nT-test for Charges Difference Between Smokers and Non-Smokers:")
smoker_charges = df_clean[df_clean['smoker'] == 'yes']['charges']
non_smoker_charges = df_clean[df_clean['smoker'] == 'no']['charges']
t_stat, p_value = stats.ttest_ind(smoker_charges, non_smoker_charges, equal_var=False)
print(f"T-statistic: {t_stat:.2f}, p-value: {p_value:.4f}")

if p_value < 0.05:
    mean_diff = smoker_charges.mean() - non_smoker_charges.mean()
    print(f"Conclusion: Smokers have significantly {'higher' if mean_diff > 0 else 'lower'} charges (${mean_diff:,.2f} difference).")
else:
    print("Conclusion: No significant difference in charges between smokers and non-smokers.")

# ANOVA Test for Charges Across BMI Categories
print("\nANOVA Test for Charges Across BMI Categories:")
bmi_groups = df_clean['bmi_category'].unique()
charges_by_bmi = [df_clean[df_clean['bmi_category'] == bmi]['charges'] for bmi in bmi_groups]
f_stat, p_value = stats.f_oneway(*charges_by_bmi)
print(f"F-statistic: {f_stat:.2f}, p-value: {p_value:.4f}")

if p_value < 0.05:
    print("Conclusion: There are significant differences in charges across BMI categories.")
else:
    print("Conclusion: No significant differences in charges across BMI categories.")

# Correlation between age and charges
pearson_corr, pearson_p = stats.pearsonr(df_clean['age'], df_clean['charges'])
print(f"\nPearson Correlation between Age and Charges: {pearson_corr:.2f} (p-value: {pearson_p:.4f})")

# Chi-square Test for Smoking and Sex
contingency_table = pd.crosstab(df_clean['sex'], df_clean['smoker'])
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
print(f"\nChi-square Test for Smoking and Sex:")
print(f"Chi2-statistic: {chi2:.2f}, p-value: {p:.4f}")

if p < 0.05:
    print("Conclusion: There is a significant association between sex and smoking status.")
else:
    print("Conclusion: No significant association between sex and smoking status.")

### 5. Feature Engineering & Preprocessing for Machine Learning

In [ ]:
# Prepare data for modeling
# Target variable: charges (we'll use both original and log-transformed for comparison)
# Features: All other relevant columns

# Select features and target
features = ['age', 'sex', 'bmi', 'children', 'smoker', 'region']
target = 'charges'
log_target = 'log_charges'

# Separate features and targets
X = df_clean[features]
y = df_clean[target]
y_log = df_clean[log_target]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
_, _, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.2, random_state=42)

# Preprocessing pipeline
numeric_features = ['age', 'bmi', 'children']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['sex', 'smoker', 'region']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Feature selection
selector = SelectKBest(score_func=f_regression, k='all')

### 6. Machine Learning Model Training & Evaluation

In [ ]:
# Define models to evaluate
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'Support Vector Regression': SVR()
}

# Evaluate each model on both original and log-transformed targets
results = {}
for name, model in models.items():
    # Create pipeline for original target
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('selector', selector),
        ('model', model)])
    
    # Create pipeline for log-transformed target
    pipeline_log = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('selector', selector),
        ('model', model)])
    
    # Fit and evaluate on original target
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Fit and evaluate on log-transformed target
    pipeline_log.fit(X_train, y_train_log)
    y_pred_log = pipeline_log.predict(X_test)
    # Convert predictions back from log scale
    y_pred_exp = np.expm1(y_pred_log)
    mse_log = mean_squared_error(y_test, y_pred_exp)
    rmse_log = np.sqrt(mse_log)
    mae_log = mean_absolute_error(y_test, y_pred_exp)
    r2_log = r2_score(y_test, y_pred_exp)
    
    # Store results
    results[name] = {
        'Original Target': {
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2
        },
        'Log Target': {
            'RMSE': rmse_log,
            'MAE': mae_log,
            'R2': r2_log
        },
        'model': pipeline,
        'model_log': pipeline_log
    }
    
    print(f"{name}:")
    print(f"  Original - RMSE: {rmse:,.2f}, MAE: {mae:,.2f}, R2: {r2:.2f}")
    print(f"  Log Trans - RMSE: {rmse_log:,.2f}, MAE: {mae_log:,.2f}, R2: {r2_log:.2f}")
    print()

# Compare model performance
results_df = pd.DataFrame.from_dict({(i,j): results[i][j] 
                                   for i in results.keys() 
                                   for j in results[i].keys() 
                                   if j in ['Original Target', 'Log Target']},
                                  orient='index')

print("\nModel Performance Comparison:")
display(results_df)

# Visualize model performance
plt.figure(figsize=(14, 6))
results_df.reset_index().pivot(index='level_0', columns='level_1', values='RMSE').plot(kind='bar')
plt.title('Model Comparison by RMSE (Lower is Better)')
plt.ylabel('Root Mean Squared Error (RMSE)')
plt.xticks(rotation=45)
plt.show()

### 7. Hyperparameter Tuning for Best Model

In [ ]:
# Tune the best performing model (Gradient Boosting with log transform in this case)
best_model_name = 'Gradient Boosting'
best_target_type = 'Log Target'
print(f"\nPerforming hyperparameter tuning for {best_model_name} ({best_target_type})...")

# Define parameter grid for Gradient Boosting
param_grid = {
    'model__n_estimators': [100, 200],
    'model__learning_rate': [0.05, 0.1],
    'model__max_depth': [3, 5],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

# Create pipeline for GridSearchCV (using log-transformed target)
pipeline_log = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('selector', selector),
    ('model', GradientBoostingRegressor(random_state=42))])

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline_log, param_grid, cv=5, 
                          scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train_log)

# Get best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Evaluate best model
y_pred_log = best_model.predict(X_test)
y_pred_exp = np.expm1(y_pred_log)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_exp))
r2 = r2_score(y_test, y_pred_exp)

print(f"\nBest Parameters: {best_params}")
print(f"Tuned Model RMSE: {rmse:,.2f}")
print(f"Tuned Model R2 Score: {r2:.2f}")

# Feature Importance from the best model
if hasattr(best_model.named_steps['model'], 'feature_importances_'):
    # Get feature names after preprocessing
    numeric_features_transformed = numeric_features
    categorical_features_transformed = best_model.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)
    feature_names = np.concatenate([numeric_features_transformed, categorical_features_transformed])
    
    # Get feature importances
    importances = best_model.named_steps['model'].feature_importances_
    indices = np.argsort(importances)[::-1]
    
    # Plot feature importances
    plt.figure(figsize=(12, 6))
    plt.title("Feature Importances")
    plt.bar(range(len(importances)), importances[indices], color="skyblue", align="center")
    plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=90)
    plt.xlim([-1, len(importances)])
    plt.tight_layout()
    plt.show()